# Conversion Code to write STELLA data and Plots to Excel in easy to read formats.
---

## Read in STELLA-Q2 that has been converted to an Excel file:
---

In [1]:
!pip install xlsxwriter

## Python libraries:

In [2]:
import pandas as pd
import xlsxwriter



## Path to data and output .xlsx files:

In [3]:
# Relative Path for STELLA Raw Excel files
data_csv_xlsx_path    = r'data.xlsx'

# path and Name of Output Excel File
output_path = 'data_output.xlsx'

## 1) Write Raw and White Card Irradiance data to **1_filtered_data.xlsx** with easy to read columns:
---

In [4]:
# Read the Excel file
df1 = pd.read_excel(data_csv_xlsx_path,index_col=False)


# Remove leading/trailing whitespaces in column names
df1.columns = df1.columns.str.strip()

# Find the column containing 'timestamp' in its name
timestamp_column_1 = [col for col in df1.columns if 'timestamp' in col.lower()][0]

# Extract wavelength values from column names and add 'nm'
new_column_headings_1 = [col.split('_')[1] + 'nm' if 'nm' not in col.split('_')[1] else col.split('_')[1] for col in df1.columns if 'irradiance_' in col and '_irradiance_uW_per_cm_squared' in col]

# Filter the columns based on your criteria
filtered_columns_1 = ['Test', 'batch', timestamp_column_1] + [col for col in df1.columns if 'irradiance_' in col and '_irradiance_uW_per_cm_squared' in col]

# Subset the DataFrame with the filtered columns
filtered_data_1 = df1[filtered_columns_1]

# Rename the columns
filtered_data_1.columns = ['Test', 'batch', timestamp_column_1] + new_column_headings_1

# Write the filtered data to a new Excel file
with pd.ExcelWriter(output_path) as writer:
    # Write the filtered data to the 'Raw' tab
    df1.to_excel(writer, sheet_name='Original Data', index=False)

    # Write the filtered data to the 'Raw' tab
    filtered_data_1.to_excel(writer, sheet_name='Raw Filtered', index=False)
   
    workbook = writer.book
    
    df_raw = filtered_data_1
    
    
     # Add a new worksheet for the Raw data plots
    chart_worksheet_raw = workbook.add_worksheet('Raw Plots')

    # Extract wavelengths and time stamps for Raw data
    wavelengths_raw = [col.split('nm')[0] for col in df_raw.columns if 'nm' in col]
    timestamps_raw = df_raw['timestamp_iso8601']

    # Write the wavelengths to the worksheet
    chart_worksheet_raw.write_row('A1', ['Timestamp'] + wavelengths_raw)

    # Write the timestamps and irradiance values to the worksheet
    for i, ts in enumerate(timestamps_raw, start=1):
        chart_worksheet_raw.write(i, 0, ts)
        chart_worksheet_raw.write_row(i, 1, df_raw.iloc[i - 1][[col + 'nm' for col in wavelengths_raw]])

    # Create a line chart for all rows in Raw data
    chart_raw = workbook.add_chart({'type': 'line'})
    for i in range(len(timestamps_raw)):
        chart_raw.add_series({
            'name': f'Row {i + 1}',
            'categories': ['Raw Plots', 1, 0, len(timestamps_raw), 0],
            'values': ['Raw Plots', 1 + i, 1, 1 + i, len(wavelengths_raw)],
        })
    chart_raw.set_x_axis({'name': 'Timestamp'})
    chart_raw.set_y_axis({'name': 'Irradiance'})
    chart_raw.set_title({'name': 'Raw Data Wavelenght Plot'})
    chart_worksheet_raw.insert_chart('E1', chart_raw)
   
    
    
    